## **Assignment - Week 8**

#### **Problem Statement**
Build a RAG Q&A chatbot using document retrieval and generative AI for intelligent response generation.

#### **Resources**
[Kaggle] https://www.kaggle.com/datasets/sonalisingh1411/loan-approval-prediction?select=Training+Dataset.csv

In [9]:
import pandas as pd

df = pd.read_csv("rag_chatbot/data/train.csv")
print(df.head())
print(df.isnull().sum())


    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [10]:
# Filling categorical columns with mode
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']:
    df[col] = df[col].fillna(df[col].mode()[0])

# Filling numerical columns with median
for col in ['LoanAmount', 'Loan_Amount_Term']:
    df[col] = df[col].fillna(df[col].median())

In [11]:
# Converting all relevant fields to strings to avoid errors
for col in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
            'Property_Area', 'Loan_Status']:
    df[col] = df[col].astype(str)


In [ ]:
def row_to_text(row):
    return (
        f"Loan ID {row['Loan_ID']} was applied by "
        f"{'a married' if row['Married'] == 'Yes' else 'an unmarried'} "
        f"{row['Gender'].lower()} "
        f"{'graduate' if row['Education'] == 'Graduate' else 'not graduate'} "
        f"with {row['Dependents']} dependents. "
        f"The applicant has an income of ₹{row['ApplicantIncome']} and "
        f"co-applicant income of ₹{row['CoapplicantIncome']}. "
        f"The loan amount requested was ₹{row['LoanAmount']} with a term of {row['Loan_Amount_Term']} months. "
        f"The credit history is {'good' if row['Credit_History'] == 1.0 else 'bad'}. "
        f"The property area is {row['Property_Area'].lower()}. "
        f"The loan was {'approved' if row['Loan_Status'] == 'Y' else 'not approved'}."
    )

df["doc_text"] = df.apply(row_to_text, axis=1)
# Sample text from the dataframe
df["doc_text"].sample(1).values[0]


'Loan ID LP002505 was applied by a married male graduate with 0 dependents. The applicant has an income of ₹4333 and co-applicant income of ₹2451.0. The loan amount requested was ₹110.0 with a term of 360.0 months. The credit history is good. The property area is urban. The loan was not approved.'